In [2]:
import ezc3d
walk_2 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/Walk-002/pose_filt_0.c3d")
hop_single = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/HopSingle-001/pose_filt_0.c3d")
cmj_3 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/CMJ-003/pose_filt_0.c3d")
triple_hop_2 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/HopTriple-002/pose_filt_0.c3d")
pistol_1 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/Pistol-001/pose_filt_0.c3d")
pistol_2 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/Pistol-002/pose_filt_0.c3d")
squat_2 = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/Squat001-000/pose_filt_0.c3d")
jog = ezc3d.c3d("/Users/harrietdray/Biodynamics/Harriet_c3d/Jog-001/pose_filt_0.c3d")


labels = squat_2['parameters']['ROTATION']['LABELS']['value'] # List of position/rotation names
# Let's explore the structure of your C3D file


print("=== C3D File Structure ===")
print("Top level keys:", list(squat_2.keys()))

print("\n=== Data Structure ===")
if 'data' in squat_2:
    print("Data keys:", list(squat_2['data'].keys()))
    
    # Check each data type
    for key in squat_2['data'].keys():
        data = squat_2['data'][key]
        print(f"{key} shape: {data.shape if hasattr(data, 'shape') else 'No shape'}")

print("\n=== Parameters Structure ===")
if 'parameters' in squat_2:
    print("Parameter groups:", list(squat_2['parameters'].keys()))
    
    # Look for rotation-related parameters
    for group in squat_2['parameters'].keys():
        if 'ROTATION' in group.upper() or 'ANALOG' in group.upper():
            print(f"\n{group} parameters:")
            if 'LABELS' in squat_2['parameters'][group]:
                labels_data = squat_2['parameters'][group]['LABELS']
                print(f"  Labels: {labels_data.get('value', 'No value key')}")

print("\n=== Your Current Labels ===")
print("Labels source:", squat_2['parameters']['ROTATION']['LABELS']['value'][:5], "...")
print("Total labels:", len(labels))

=== C3D File Structure ===
Top level keys: ['header', 'parameters', 'data']

=== Data Structure ===
Data keys: ['points', 'meta_points', 'analogs', 'rotations']
points shape: (4, 20, 547)
meta_points shape: No shape
analogs shape: (1, 0, 0)
rotations shape: (4, 4, 19, 547)

=== Parameters Structure ===
Parameter groups: ['POINT', 'ANALOG', 'FORCE_PLATFORM', 'MANUFACTURER', 'THEIA3D', 'PROCESSING', 'TRIAL', 'ROTATION', 'EVENT', 'DATA_BLOCKS', 'EZC3D']

ANALOG parameters:
  Labels: []

ROTATION parameters:
  Labels: ['worldbody_4X4', 'pelvis_4X4', 'l_thigh_4X4', 'l_shank_4X4', 'l_foot_4X4', 'l_toes_4X4', 'r_thigh_4X4', 'r_shank_4X4', 'r_foot_4X4', 'r_toes_4X4', 'head_4X4', 'torso_4X4', 'l_uarm_4X4', 'l_larm_4X4', 'l_hand_4X4', 'r_uarm_4X4', 'r_larm_4X4', 'r_hand_4X4', 'pelvis_shifted_4X4']

=== Your Current Labels ===
Labels source: ['worldbody_4X4', 'pelvis_4X4', 'l_thigh_4X4', 'l_shank_4X4', 'l_foot_4X4'] ...
Total labels: 19


In [3]:
# Let's debug the actual structure of your rotation data
print("=== DEBUGGING ROTATION DATA STRUCTURE ===")
rotation_data = squat_2['data']['rotations']
print(f"Rotation data shape: {rotation_data.shape}")
print(f"Data type: {type(rotation_data)}")

# Let's see what the first few values look like
print(f"First 20 values of rotation_data[:, 0, 0]: {rotation_data[:20, 0, 0]}")

# Check if the data might be stored differently
print(f"Min value in rotation data: {rotation_data.min()}")
print(f"Max value in rotation data: {rotation_data.max()}")

# Let's look at the labels more carefully
print(f"\nLabels (first 10): {labels[:10]}")
print(f"Total labels: {len(labels)}")

# Let's try to understand the data better
print(f"Shape breakdown:")
print(f"  Dimension 0 (should be 16 for 4x4 matrix): {rotation_data.shape[0]}")
print(f"  Dimension 1 (frames): {rotation_data.shape[1]}")  
print(f"  Dimension 2 (joints): {rotation_data.shape[2]}")

# The issue might be that it's not 16 elements per matrix
# Let's check if each "joint" actually contains multiple matrices
single_joint_data = rotation_data[:, 0, 0]  # All data for first joint, first frame
print(f"\nSingle joint data size: {len(single_joint_data)}")
print(f"This might represent {len(single_joint_data)/16} matrices of 4x4 each")

=== DEBUGGING ROTATION DATA STRUCTURE ===
Rotation data shape: (4, 4, 19, 547)
Data type: <class 'numpy.ndarray'>
First 20 values of rotation_data[:, 0, 0]: [[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Min value in rotation data: -279.69354248046875
Max value in rotation data: 1482.078125

Labels (first 10): ['worldbody_4X4', 'pelvis_4X4', 'l_thigh_4X4', 'l_shank_4X4', 'l_foot_4X4', 'l_toes_4X4', 'r_thigh_4X4', 'r_shank_4X4', 'r_foot_4X4', 'r_toes_4X4']
Total labels: 19
Shape breakdown:
  Dimension 0 (should be 16 for 4x4 matrix): 4
  Dimension 1 (frames): 4
  Dimension 2 (joints): 19

Single joint data size: 4
This might represent 0.25 matrices of 4x4 each
